In [1]:
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras import optimizers
from keras.layers.advanced_activations import ELU, PReLU, LeakyReLU
from keras.layers import Dense, Dropout, Activation, Flatten

Using TensorFlow backend.


In [2]:
model = Sequential()
model.add(Conv2D(8, (3, 3), padding='same',
                 input_shape=(513, 800, 3)))
model.add(Activation('relu'))
model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dense(10))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(57))
model.add(Activation('softmax'))

model.summary()

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 513, 800, 8)       224       
_________________________________________________________________
activation_1 (Activation)    (None, 513, 800, 8)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 511, 798, 8)       584       
_________________________________________________________________
activation_2 (Activation)    (None, 511, 798, 8)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 255, 399, 8)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 255, 399, 8)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 255, 399, 16)      1168      
__________

In [3]:
model.load_weights('my_model_weights.h5')

In [4]:
import subprocess
import matplotlib.pyplot
import os
import matplotlib.pyplot as plt
rootdir = './train/'

## Load new people for dataset

In [5]:
spectograms = []
spect_read = []
spectograms_ids = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith('png'): 
            try:
                x = plt.imread(subdir+'/'+file)
            except:
                continue
            if str(x.shape) == '(513, 800, 3)': 
                spect_read.append(x)
                #print(subdir) 
                name = subdir.replace(rootdir, '')
                #print(name)
                #name = name.replace('/spects', "")
                spectograms_ids.append(name)
                spectograms.append(file)
x_train = spect_read
y_train = spectograms_ids
y_train

[]

In [60]:
rootdir = '/home/ubuntu/new_ppl/test/'
spectograms = []
spect_read = []
spectograms_ids = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith('png'): 
            try:
                x = plt.imread(subdir+'/'+file)
            except:
                continue
            if str(x.shape) == '(513, 800, 3)': 
                spect_read.append(x)
                name = subdir.replace(rootdir, '')
                #name = name.replace('/spects', "")
                spectograms_ids.append(name)
                spectograms.append(file)
x_test = spect_read
y_test = spectograms_ids
y_test

['christen',
 'christen',
 'christen',
 'christen',
 'christen',
 'christen',
 'christen',
 'tushar',
 'tushar',
 'tushar',
 'tushar',
 'tushar',
 'tushar',
 'tushar',
 'sriraj',
 'sriraj',
 'sriraj',
 'sriraj',
 'sriraj',
 'sriraj',
 'sriraj']

## One hot encode

In [61]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
y_temp_train = y_train
encoder.fit(y_temp_train)
encoded_Y = encoder.transform(y_temp_train)
dummy_y = np_utils.to_categorical(encoded_Y)

y_temp2_train = y_test
encoder.fit(y_temp2_train)
encoded_Y = encoder.transform(y_temp2_train)
dummy2_y = np_utils.to_categorical(encoded_Y)

In [64]:
dummy2_y

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.]])

## Transfer Learning

In [65]:
from keras.models import Model
model2 = Model(inputs=model.input, outputs=model.get_layer('flatten_1').output)

In [66]:
svm_x_train = []
svm_y_train = []
for i in range(len(x_train)):
    x_1 = np.expand_dims(x_train[i], axis=0)
    flatten_2_features = model2.predict(x_1)
    svm_x_train.append(flatten_2_features)
    svm_y_train.append(dummy_y[i])

In [67]:
svm_y_train

[array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  0.,  1.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 0.,  1.,  0.])]

In [68]:
from sklearn import svm
svm_x_train = np.array(svm_x_train)
clf = svm.SVC(kernel='rbf', class_weight='balanced')
dataset_size = len(svm_x_train)
svm_x_train = np.array(svm_x_train).reshape(dataset_size,-1)
svm_y_train = np.array(svm_y_train)
svm_y_train = [np.where(r==1)[0][0] for r in svm_y_train]


In [69]:
clf.fit(svm_x_train, svm_y_train)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [72]:
svm_x_test = []
svm_y_test = []
for i in range(len(x_test)):
    x_1 = np.expand_dims(x_test[i], axis=0)
    #x_1 = preprocess_input(x_1)
    flatten_2_features = model2.predict(x_1)
    svm_x_test.append(flatten_2_features)
    svm_y_test.append(dummy2_y[i])
svm_x_test = np.array(svm_x_test)

In [73]:
dataset_size = len(svm_x_test)
svm_x_test = np.array(svm_x_test).reshape(dataset_size,-1)
svm_y_test = [np.where(r==1)[0][0] for r in svm_y_test]

In [74]:
print(svm_y_test)

[0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1]


In [75]:
print(clf.predict(svm_x_test))

[0 0 0 0 0 0 0 2 2 2 2 2 2 2 1 1 2 1 1 1 1]


In [77]:
from sklearn.metrics import accuracy_score
accuracy_score(svm_y_test, clf.predict(svm_x_test))

0.95238095238095233